## Part 0: Setup and Installation

In [15]:
#@title Step 1:  Mount Google drive, setup project paths, uv and dependancies
import os
from huggingface_hub import HfApi, list_repo_files
from IPython.utils.text import list_strings
from google.colab import files
import pathlib
import shutil
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/Colab Notebooks/MSAAI/AAI_521/Project"
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")

os.makedirs(PROJECT_ROOT, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

print("Project Paths:")
print(f" Project Root: {PROJECT_ROOT}")
print(f" Data Directory: {DATA_DIR}")
print(f" Models Directory: {MODELS_DIR}")

!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"
!uv --version
!uv pip install huggingface-hub --system

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project Paths:
 Project Root: /content/drive/MyDrive/Colab Notebooks/MSAAI/AAI_521/Project
 Data Directory: /content/drive/MyDrive/Colab Notebooks/MSAAI/AAI_521/Project/data
 Models Directory: /content/drive/MyDrive/Colab Notebooks/MSAAI/AAI_521/Project/models
downloading uv 0.9.11 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
uv 0.9.11
Using Python 3.12.12 environment at: /usr
Audited 1 package in 90ms


In [ ]:
#@title Step 2:  Check if Models exist on Hugging Face Hub

REPO_ID = "d2j666/asl-efficientnets"
REQUIRED_FILES = ["efficientnetb4_asl.h5", "efficientnetb7_asl.h5", "efficientnetb9_asl.h5"]
SKIP_TRAINING = False

print("Checking if models exist on Hugging Face Hub...")

try:
  api = HfApi()
  repo = api.repo_info(REPO_ID)
  files = list_repo_files(repo_id=REPO_ID, repo_type="model")

  existing_models = [f for f in REQUIRED_FILES if f in files]
  missing_models = [f for f in REQUIRED_FILES if f not in files]

  if len(existing_models) == len(REQUIRED_FILES):
    print(f" All required models exist on Hugging Face Hub repository {REPO_ID}")

    for model in existing_models:
      try:
          print(f"   Model {model} found.")
      except:
          print(f"   Model {model} not found.")

    SKIP_TRAINING = True
    
  elif len(existing_models) < len(REQUIRED_FILES):
    print(f"  Some models exist on Hugging Face Hub repository {REPO_ID}, but not all required models.")
    print(f"   Existing models: {existing_models}")
    print(f"   Missing models: {missing_models}")
    SKIP_TRAINING = False

except Exception as e:
    error_msg = str(e).lower()
    SKIP_TRAINING = False

print(f"\n Skip Training? {SKIP_TRAINING}")

In [14]:
#@title Step 2:  Check GPU Availability

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB7, EfficientNetV2L
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input

print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))

if len(tf.config.list_physical_devices('GPU')) == 0:
  raise Exception("Go to Runtime → Change runtime type → Select GPU (A100 if available)")

TensorFlow version: 2.19.0
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
#@title Step 3: Download Kaggle dataset directly to Colab
from google.colab import files

# Check if kaggle.json already exists
kaggle_dir = pathlib.Path.home() / ".kaggle"
kaggle_json_path = kaggle_dir / "kaggle.json"

if not kaggle_json_path.exists():
  print("📋 Kaggle credentials not found")
  print("   Please upload your kaggle.json file:\n")
  uploaded = files.upload()

  if "kaggle.json" not in uploaded:
    raise Exception("❌ Could not upload kaggle.json")
  else:
    kaggle_dir.mkdir(exist_ok=True)
    shutil.move("kaggle.json", kaggle_json_path)
    os.chmod(kaggle_json_path, 0o600)
    print("✅ Kaggle credentials configured!")
else:
  print("✅ Kaggle credentials already exist")

print("\n🔍 Testing Kaggle API connection...")
!kaggle datasets list -s "asl alphabet" | head -3

# Define paths
LOCAL_DATA_PATH = "/content/asl_alphabet"
TRAIN_DATA_PATH = os.path.join(LOCAL_DATA_PATH, "asl_alphabet_train", "asl_alphabet_train")
TEST_DATA_PATH = os.path.join(LOCAL_DATA_PATH, "asl_alphabet_test", "asl_alphabet_test")

# Download data directly to Colab local storage (fastest option)
if os.path.exists(TRAIN_DATA_PATH) and len(os.listdir(TRAIN_DATA_PATH)) == 29:
  print(f"\n✅ Dataset already in Colab local storage")
  print(f"   Classes found: {len(os.listdir(TRAIN_DATA_PATH))}")
else:
  print("\n📥 Downloading ASL dataset from Kaggle to Colab...")
  print("   Size: ~1.1 GB | Extracting directly to local storage")
  print("   ⚡ Direct download is faster than copying from Drive\n")
  !kaggle datasets download -d grassknoted/asl-alphabet -p {LOCAL_DATA_PATH} --unzip
  print("\n✅ Dataset downloaded and extracted!")

print(f"\n📊 Dataset Ready:")
print(f"   Train path: {TRAIN_DATA_PATH}")
print(f"   Test path: {TEST_DATA_PATH}")
print(f"   Classes: {os.listdir(TRAIN_DATA_PATH)[:10]} ... (29 total)")

# Optional: Copy to Drive for backup (uncomment if needed)
# ASL_DATA_ROOT = os.path.join(DATA_DIR, "asl_alphabet")
# if not os.path.exists(ASL_DATA_ROOT):
#   print(f"\n💾 Backing up dataset to Drive...")
#   shutil.copytree(LOCAL_DATA_PATH, ASL_DATA_ROOT)
#   print(f"   ✅ Saved to: {ASL_DATA_ROOT}")

## Part 1:  Pipeline Setup, Model Building, Training & Testing

In [ ]:
#@title Step 1:  Setup Data Generators
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 64  # Use 32 if GPU memory limited
NUM_CLASSES = 29
AUTOTUNE = tf.data.AUTOTUNE

# Create datasets using modern API
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  TRAIN_DATA_PATH,
  label_mode="categorical",
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE,
  validation_split=0.2,
  subset="training",
  seed=42
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  TRAIN_DATA_PATH,
  label_mode="categorical",
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE,
  validation_split=0.2,
  subset="validation",
  seed=42
)

class_names = train_ds.class_names

# Performance optimizations
train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# Augmentation
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.05),      # ~18 degrees
  layers.RandomZoom(0.1),
  layers.RandomTranslation(0.1, 0.1),
])

print(f"\n Data pipeline ready!")
print(f"  Training batches: {len(train_ds)}")
print(f"  Validation batches: {len(val_ds)}")
print(f"  Classes: {class_names[:5]} ... ({NUM_CLASSES} total)")

Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.

 Data pipeline ready!
  Training batches: 1088
  Validation batches: 272
  Classes: ['A', 'B', 'C', 'D', 'E'] ... (29 total)


In [ ]:
#@title Step 1a: Optional callbacks

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("   Available: EarlyStopping, ReduceLROnPlateau, ModelCheckpoint")
print("   Note: Not used by default, but ready if you want to add them")

In [ ]:
#@title Step 2:  Build EfficientNet Model
def build_efficientnet_model(model_name="B4", num_classes=29):
  """
  Build EfficientNet model with 2-stage training capability.

  Args:
      model_name: "B4", "B7", or "B9"
      num_classes: Number of output classes (29 for ASL)

  Returns:
      Tuple of (model, base_model) ready for 2-stage training
  """
  print(f"\n{'='*60}")
  print(f"Building EfficientNet{model_name} model...")
  print(f"{'='*60}")

  # Load pre-trained base model
  if model_name == "B4":
      base_model = EfficientNetB4(
        include_top=False,
        weights="imagenet",
        input_shape=IMG_SIZE + (3,)
      )
  elif model_name == "B7":
      base_model = EfficientNetB7(
        include_top=False,
        weights="imagenet",
        input_shape=IMG_SIZE + (3,)
      )
  elif model_name == "B9":
      base_model = EfficientNetV2L(
        include_top=False,
        weights="imagenet",
        input_shape=IMG_SIZE + (3,)
      )
  else:
      raise ValueError(f"Unknown model name: {model_name}")

  # Build model with augmentation
  inputs = layers.Input(shape=IMG_SIZE + (3,))
  x = data_augmentation(inputs)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  x = base_model(x, training=False)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dropout(0.3)(x)
  outputs = layers.Dense(num_classes, activation="softmax")(x)

  model = models.Model(inputs, outputs)

  print(f"\n Model built successfully!")
  print(f"   Total parameters: {model.count_params():,}")

  return model, base_model

In [ ]:
#@title Step 3: Override SKIP_TRAINING (Optional - Set to False to force training)
# Uncomment the line below to force training even if models exist
# SKIP_TRAINING = False

print(f" Skip Training? {SKIP_TRAINING}")
if SKIP_TRAINING:
  print("    Training will be skipped (models already exist)")
  print("    To force retraining, uncomment the line above")
else:
  print("   🔄 Training will proceed")
  print("   ⏱️  Expected time: ~2 hours with GPU")

In [ ]:
#@title Step 3a: Train EfficientNetB4 (2-Stage Fine-Tuning)
# Stage 1: Freeze backbone, train head (5 epochs)
# Stage 2: Fine-tune top layers (5 epochs)

if SKIP_TRAINING:
  print("\n" + "="*60)
  print("⏭️  SKIPPING EFFICIENTNETB4 TRAINING")
  print("="*60)
  print("\n Models already exist in HuggingFace Hub")
  print(" Run Step 17A to download them to your Drive")
else:
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB4 - STAGE 1: WARM-UP")
  print("="*60 + "\n")

  # Build model
  model_b4, base_b4 = build_efficientnet_model("B4", num_classes=NUM_CLASSES)

  # Stage 1: Freeze backbone, train classification head
  base_b4.trainable = False

  model_b4.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
  )

  # Train Stage 1
  history_b4_stage1 = model_b4.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )

  print(f"\n Stage 1 complete! Val accuracy: {history_b4_stage1.history['val_accuracy'][-1]:.4f}")

  # Stage 2: Fine-tune top layers
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB4 - STAGE 2: FINE-TUNING")
  print("="*60 + "\n")

  base_b4.trainable = True

  # Freeze lower layers, train top ~150 layers
  for layer in base_b4.layers[:-150]:
      layer.trainable = False

  model_b4.compile(
      optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate!
      loss="categorical_crossentropy",
      metrics=["accuracy"]
  )

  # Train Stage 2
  history_b4_stage2 = model_b4.fit(
      train_ds,
      validation_data=val_ds,
      epochs=5
  )

  # Save model
  b4_save_path = os.path.join(MODELS_DIR, 'efficientnetb4_asl.h5')
  model_b4.save(b4_save_path)

  best_val_acc = max(history_b4_stage2.history['val_accuracy'])
  print(f"\n EfficientNetB4 training complete!")
  print(f"   Saved to: {b4_save_path}")
  print(f"   Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
#@title Step 3b: Train EfficientNetB7 (2-Stage Fine-Tuning)
# Stage 1: Freeze backbone, train head (5 epochs)
# Stage 2: Fine-tune top layers (5 epochs)

if SKIP_TRAINING:
  print("\n" + "="*60)
  print("⏭️  SKIPPING EFFICIENTNETB7 TRAINING")
  print("="*60)
  print("\n Models already exist in HuggingFace Hub")
else:
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB7 - STAGE 1: WARM-UP")
  print("="*60 + "\n")

  # Build model
  model_b7, base_b7 = build_efficientnet_model("B7", num_classes=NUM_CLASSES)

  # Stage 1: Freeze backbone, train classification head
  base_b7.trainable = False

  model_b7.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
  )

  # Train Stage 1
  history_b7_stage1 = model_b7.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )

  print(f"\n Stage 1 complete! Val accuracy: {history_b7_stage1.history['val_accuracy'][-1]:.4f}")

  # Stage 2: Fine-tune top layers
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB7 - STAGE 2: FINE-TUNING")
  print("="*60 + "\n")

  base_b7.trainable = True

  # Freeze lower layers, train top ~150 layers
  for layer in base_b7.layers[:-150]:
      layer.trainable = False

  model_b7.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate!
    loss="categorical_crossentropy",
    metrics=["accuracy"]
  )

  # Train Stage 2
  history_b7_stage2 = model_b7.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )

  # Save model
  b7_save_path = os.path.join(MODELS_DIR, 'efficientnetb7_asl.h5')
  model_b7.save(b7_save_path)

  best_val_acc = max(history_b7_stage2.history['val_accuracy'])
  print(f"\n EfficientNetB7 training complete!")
  print(f"   Saved to: {b7_save_path}")
  print(f"   Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
#@title Step 3c: Train EfficientNetB9 (2-Stage Fine-Tuning)
# Stage 1: Freeze backbone, train head (5 epochs)
# Stage 2: Fine-tune top layers (5 epochs)

if SKIP_TRAINING:
  print("\n" + "="*60)
  print("⏭️  SKIPPING EFFICIENTNETB9 TRAINING")
  print("="*60)
  print("\n Models already exist in HuggingFace Hub")
else:
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB9 - STAGE 1: WARM-UP")
  print("="*60 + "\n")

  # Build model
  model_b9, base_b9 = build_efficientnet_model("B9", num_classes=NUM_CLASSES)

  # Stage 1: Freeze backbone, train classification head
  base_b9.trainable = False

  model_b9.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
  )

  # Train Stage 1
  history_b9_stage1 = model_b9.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )

  print(f"\n Stage 1 complete! Val accuracy: {history_b9_stage1.history['val_accuracy'][-1]:.4f}")

  # Stage 2: Fine-tune top layers
  print("\n" + "="*60)
  print("TRAINING EFFICIENTNETB9 - STAGE 2: FINE-TUNING")
  print("="*60 + "\n")

  base_b9.trainable = True

  # Freeze lower layers, train top ~150 layers
  for layer in base_b9.layers[:-150]:
      layer.trainable = False

  model_b9.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate!
    loss="categorical_crossentropy",
    metrics=["accuracy"]
  )

  # Train Stage 2
  history_b9_stage2 = model_b9.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )

  # Save model
  b9_save_path = os.path.join(MODELS_DIR, 'efficientnetb9_asl.h5')
  model_b9.save(b9_save_path)

  best_val_acc = max(history_b9_stage2.history['val_accuracy'])
  print(f"\n EfficientNetB9 training complete!")
  print(f"   Saved to: {b9_save_path}")
  print(f"   Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
#@title Step 4: Training Results Comparison

if SKIP_TRAINING:
  print("\n No training occurred (models already exist)")
  print("   Models are available on HuggingFace Hub")
else:
  import matplotlib.pyplot as plt
  import pandas as pd

  def combine_histories(h1, h2):
    combined = {}
    for key in h1.history.keys():
      combined[key] = h1.history[key] + h2.history[key]
    return combined

  history_b4_full = combine_histories(history_b4_stage1, history_b4_stage2)
  history_b7_full = combine_histories(history_b7_stage1, history_b7_stage2)
  history_b9_full = combine_histories(history_b9_stage1, history_b9_stage2)

  results = {
      "Model": ["EfficientNetB4", "EfficientNetB7", "EfficientNetB9"],
      "Stage 1 Val Acc": [
          f"{history_b4_stage1.history['val_accuracy'][-1]:.4f}",
          f"{history_b7_stage1.history['val_accuracy'][-1]:.4f}",
          f"{history_b9_stage1.history['val_accuracy'][-1]:.4f}"
      ],
      "Final Val Acc": [
          f"{history_b4_full['val_accuracy'][-1]:.4f}",
          f"{history_b7_full['val_accuracy'][-1]:.4f}",
          f"{history_b9_full['val_accuracy'][-1]:.4f}"
      ],
      "Best Val Acc": [
          f"{max(history_b4_full['val_accuracy']):.4f}",
          f"{max(history_b7_full['val_accuracy']):.4f}",
          f"{max(history_b9_full['val_accuracy']):.4f}"
      ]
  }

  df_results = pd.DataFrame(results)
  print("\n" + "="*70)
  print("MODEL COMPARISON SUMMARY (2-STAGE TRAINING)")
  print("="*70)
  print(df_results.to_string(index=False))
  print("="*70)

  # Plot training curves
  fig, axes = plt.subplots(1, 3, figsize=(18, 5))

  for idx, (history, name) in enumerate([
    (history_b4_full, "EfficientNetB4"),
    (history_b7_full, "EfficientNetB7"),
    (history_b9_full, "EfficientNetB9")
  ]):
    ax = axes[idx]
    epochs = range(1, len(history['accuracy']) + 1)
    ax.plot(epochs, history['accuracy'], label='Train Acc', linewidth=2)
    ax.plot(epochs, history['val_accuracy'], label='Val Acc', linewidth=2)
    ax.axvline(x=5, color='red', linestyle='--', label='Stage 1→2')
    ax.set_title(f"{name}\n2-Stage Training", fontsize=14, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)

  plt.tight_layout()
  chart_path = os.path.join(MODELS_DIR, 'training_comparison.png')
  plt.savefig(chart_path, dpi=150, bbox_inches='tight')
  print(f"\n Comparison chart saved to: {chart_path}")
  plt.show()

In [ ]:
#@title Step 5: Test Models with Sample Predictions

if SKIP_TRAINING:
  print("\n" + "="*60)
  print("  SKIPPING MODEL TESTING")
  print("="*60)
  print("\n  Models were not trained in this session")
  print(" To test predictions:")
  print("   1. Set SKIP_TRAINING = False and run training cells, OR")
  print("   2. Load models from Drive/HuggingFace Hub first")
else:
  def test_prediction(model, img_path, model_name):
      """Test a single image prediction."""
      img = image.load_img(img_path, target_size=(224, 224))
      img_array = image.img_to_array(img)
      img_array = preprocess_input(img_array)
      img_array = np.expand_dims(img_array, axis=0)

      preds = model.predict(img_array, verbose=0)[0]
      top_idx = np.argmax(preds)

      print(f"{model_name:20} → {class_names[top_idx]:10} (confidence: {preds[top_idx]:.3f})")

  print("Testing Model Predictions")

  for test_num in range(3):
      test_class = random.choice(class_names)
      test_class_dir = os.path.join(TRAIN_DATA_PATH, test_class)
      test_img = random.choice(os.listdir(test_class_dir))
      test_img_path = os.path.join(test_class_dir, test_img)

      print(f"Test {test_num + 1}: True label = '{test_class}'")
      print("-" * 60)

      test_prediction(model_b4, test_img_path, "EfficientNetB4")
      test_prediction(model_b7, test_img_path, "EfficientNetB7")
      test_prediction(model_b9, test_img_path, "EfficientNetB9")

      print("\nSample image:")
      display(Image.open(test_img_path).resize((150, 150)))
      print("\n" + "="*60 + "\n")

In [ ]:
#@title Step 6: Upload Models to HuggingFace Hub

if SKIP_TRAINING:
  print("\n" + "="*60)
  print(" SKIPPING MODEL UPLOAD")
  print("="*60)
  print("\n  No models to upload (training was skipped)")
  print(f"    Models already exist at: https://huggingface.co/{REPO_ID}")
else:
  print("\n Please ensure your HuggingFace token is set as a Colab secret (HF_TOKEN)")
  print("    Get it from: https://huggingface.co/settings/tokens (type: Write)\n")

  api = HfApi()
  REPO_ID = "d2j666/asl-efficientnets"
  
  try:
      api.create_repo(repo_id=REPO_ID, exist_ok=True, private=True)
      print(f" Repository '{REPO_ID}' created or already exists.")
  except Exception as e:
      print(f" An error occurred: {e}")
  
  print(f"\n Uploading models to: {REPO_ID}\n")

  model_files = [
      (os.path.join(MODELS_DIR, 'efficientnetb4_asl.h5'), 'efficientnetb4_asl.h5'),
      (os.path.join(MODELS_DIR, 'efficientnetb7_asl.h5'), 'efficientnetb7_asl.h5'),
      (os.path.join(MODELS_DIR, 'efficientnetb9_asl.h5'), 'efficientnetb9_asl.h5')
  ]

  upload_count = 0
  for local_path, repo_path in model_files:
      if os.path.exists(local_path):
          file_size_mb = os.path.getsize(local_path) / (1024 * 1024)
          print(f" Uploading {repo_path} ({file_size_mb:.1f} MB)...")

          try:
              api.upload_file(
                  path_or_fileobj=local_path,
                  path_in_repo=repo_path,
                  repo_id=REPO_ID,
                  repo_type="model"
              )
              print(f"    {repo_path} uploaded!\n")
              upload_count += 1
          except Exception as e:
              print(f"    Error: {e}\n")
      else:
          print(f"  {repo_path} not found at {local_path}\n")

  if upload_count == len(model_files):
      print(" ALL MODELS UPLOADED!")
  else:
      print(f"  PARTIAL UPLOAD ({upload_count}/{len(model_files)} successful)")
  print(f"\n View at: https://huggingface.co/{REPO_ID}")